In [ ]:
!pip install -U openai-whisper

In [ ]:
import pandas as pd
import requests
import subprocess
import json
import whisper

# Read CSV as a DataFrame
csv_file_path = 'tagesschau_sign_language_video_links.csv'
df = pd.read_csv(csv_file_path)
df

In [ ]:
# Select medium quality video url
video_url = df['webm'].iloc[-1]
video_url

In [ ]:
# Function to download a file from a URL
def download_file(url, file_name):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(file_name, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)
        print(f"Downloaded {file_name}")
    else:
        print(f"Failed to download {file_name}. Status code: {response.status_code}")

# Download the video file
video_file_name = video_url.split("/")[-1]
download_file(video_url, video_file_name)

In [ ]:
command = [
    'ffmpeg',
    '-i', video_file_name,
    '-ar', '16000',
    '-ac', '1',
    'audio.wav'
]

subprocess.run(command, check=True)

In [ ]:
# Load the large-v3-turbo model (or replace with the correct model if naming differs)
model = whisper.load_model("large-v3-turbo")

# Transcribe the audio file
result = model.transcribe("audio.wav", language="de", word_timestamps=True)

# Extract word-level timestamps
word_segments = result.get("segments")
word_segments

In [ ]:
# TODO: Replace with save in Google Cloud
with open(video_file_name + ".json", "w") as f:
  json.dump(word_segments, f)